In [1]:
import os
import datetime
import random # random.seed(42)
import warnings
import numpy as np # np.random.seed(42)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize,LinearSegmentedColormap

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import sklearn.exceptions
from sklearn.utils import class_weight
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

from tensorflow.config.experimental import enable_op_determinism # enable_op_determinism()
from tensorflow.random import set_seed # set_seed(42)
from tensorflow.python.client import device_lib

from keras import backend as K
from keras.utils import load_img, img_to_array, set_random_seed # set_random_seed(42)
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import Input, GlobalAveragePooling2D, ZeroPadding2D
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import applications
finish_sound = "afplay /Users/mehmet/Documents/vs-code/winsquare.mp3"
# !conda install -y -n ml ipykernel=6.23.2 numpy==1.24.0 matplotlib=3.7.1 pandas=2.0.2 seaborn=0.12.1 scikit-learn=1.3.2 tensorflow=2.11.1
# !jupyter nbconvert --to html skin-cancer-cnn.ipynb
[x.name for x in device_lib.list_local_devices()]

['/device:CPU:0']

In [2]:
# Train and test data paths
train_path = "dataverse_files/HAM10000_images_pca"
test_path = "dataverse_files/ISIC2018_Task3_Test_Images"

# Read the data
df = pd.read_csv('dataverse_files/HAM10000_metadata.csv')
df_test = pd.read_csv('dataverse_files/ISIC2018_Task3_Test_GroundTruth.csv')

# Delete df and vasc classes
df = df[df['dx'] != 'vasc']
df = df[df['dx'] != 'df']
df_test = df_test[df_test['dx'] != 'vasc']
df_test = df_test[df_test['dx'] != 'df']

labels = df['dx'].sort_values().unique()

# Add .jpg to image_id column
df['image_id'] = df['image_id'].astype(str) + '.jpg'
df_test['image_id'] = df_test['image_id'].astype(str) + '.jpg'

# Drop unused columns
# df=df.drop(['lesion_id',  'dx_type', 'age', 'sex', 'localization', 'dataset'], axis=1)
# df_test=df_test.drop(['lesion_id',  'dx_type', 'age', 'sex', 'localization', 'dataset'], axis=1)
df=df.drop(['lesion_id', 'dataset'], axis=1)
df_test=df_test.drop(['lesion_id', 'dataset'], axis=1)

# 'ISIC_0035068.jpg' is missing in the test set file, lets remove it from test set dataframe
df_test = df_test[df_test['image_id'] != 'ISIC_0035068.jpg']

print(labels,'\n')

df.sort_values(by=['image_id'], inplace=True)
df.reset_index(inplace=True, drop=True)
df_test.sort_values(by=['image_id'], inplace=True)
df_test.reset_index(inplace=True, drop=True)

train_df, val_df=train_test_split(df, train_size=0.9, shuffle=True, random_state=123, stratify=df['dx'])
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)
test_df = df_test.copy().sample(frac=1, random_state=123).reset_index(drop=True) # shuffle test set

['akiec' 'bcc' 'bkl' 'mel' 'nv'] 



In [3]:
#To use augmented data
x = '_3'
train_path = 'dataverse_files/HAM10000_images_pca_augmented'+x
train_df = pd.read_csv('dataverse_files/HAM10000_metadata_augmented'+x+'_train.csv')
val_df = pd.read_csv('dataverse_files/HAM10000_metadata_augmented'+x+'_val.csv')

In [4]:
for label in labels:
    list1 = len(train_df[train_df['dx'] == label]), len(val_df[val_df['dx'] == label]), len(test_df[test_df['dx'] == label])
    space = ' '
    print(label,(5-len(label))*space ,list1)
    
train_df.shape, val_df.shape, test_df.shape

akiec  (294, 33, 43)
bcc    (463, 51, 93)
bkl    (989, 110, 217)
mel    (1002, 111, 171)
nv     (6034, 671, 908)


((8782, 6), (976, 6), (1432, 6))

In [5]:
# Deleting extra images from the dataset
# np.random.seed(42)
# grouped_train = train_df.groupby('dx')
# grouped_val = val_df.groupby('dx')
# for label in labels:
#     drop_indices_train = []
    
#     if grouped_train.get_group(label).shape[0] > 1002:
#         remove_n = grouped_train.get_group(label).shape[0] - 1000 # number of images in the smallest class
#         drop_indices_train = np.random.choice(grouped_train.get_group(label).index, remove_n, replace=False)
#         train_df = train_df.drop(drop_indices_train) # drop extra images from original dataset
        
#     if grouped_val.get_group(label).shape[0] > 111:
#         remove_n = grouped_val.get_group(label).shape[0] - 100 # number of images in the smallest class
#         drop_indices_train = np.random.choice(grouped_val.get_group(label).index, remove_n, replace=False)
#         val_df = val_df.drop(drop_indices_train) # drop extra images from original dataset

# for label in labels:
#     list1 = len(train_df[train_df['dx'] == label]), len(val_df[val_df['dx'] == label]), len(test_df[test_df['dx'] == label])
#     space = ' '
#     print(label,(5-len(label))*space ,list1)
    
# train_df.shape, val_df.shape, test_df.shape

In [6]:
rescale=1./255
color_mode = 'rgb'
target_size = (32, 32)
batch_size  = 64
# 600 x 450
train_data_np = np.array([img_to_array(load_img(train_path+'/'+img, target_size=target_size)) for img in train_df['image_id'].values.tolist()])

datagen = ImageDataGenerator(rescale=rescale,
                            featurewise_center=True,
                            featurewise_std_normalization=True)
datagen.fit(train_data_np)
train_set = datagen.flow_from_dataframe(train_df,
                                        directory=train_path,
                                        x_col="image_id",
                                        y_col="dx",
                                        color_mode=color_mode,
                                        target_size=target_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=False
                                        )

val_set = datagen.flow_from_dataframe(val_df,
                                      directory=train_path,
                                      x_col="image_id",
                                      y_col="dx",
                                      color_mode=color_mode,
                                      target_size=target_size,
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                      shuffle=False
                                      )
test_set = datagen.flow_from_dataframe(test_df,
                                       directory=test_path,
                                       x_col="image_id",
                                       y_col="dx",
                                       color_mode=color_mode,
                                       target_size=target_size,
                                       batch_size=batch_size,
                                       class_mode='categorical',
                                       shuffle=False
                                       )

Found 8782 validated image filenames belonging to 5 classes.
Found 976 validated image filenames belonging to 5 classes.
Found 1432 validated image filenames belonging to 5 classes.


In [8]:
def loss_plot(model, history, now, save=True):
    # convert the history.history dict to a pandas DataFrame:
    if type(history) is not pd.DataFrame:
        history = pd.DataFrame(history)
    if save == True:
        hist_csv_file = f'model-comparison/{now}/history.csv'
        with open(hist_csv_file, mode='w') as f:
            history.to_csv(f) 
    epochs = range(1, history.shape[0]+1)
    plt.figure(figsize=(5, 2))
    plt.plot(epochs, history['accuracy'], label='Accuracy')
    plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    max_val_acc_epoch = np.argmax(history['val_accuracy']) + 1
    max_val_acc = history['val_accuracy'][max_val_acc_epoch-1]
    label='Best Epoch = '+str(max_val_acc_epoch)+'\nVal. Acc. = '+str((max_val_acc*100).round(2))+ '%'
    plt.plot(max_val_acc_epoch, max_val_acc, 'ro', label=label)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xlim([0, history.shape[0]+0.1])
    plt.ylim([0.5, 1])
    plt.title('Training and Validation Accuracy')
    plt.legend(loc='upper left')
    if save == True:
        plt.savefig(f'model-comparison/{now}/val-acc.png')
        np.savetxt('model-comparison/{}/{}.txt'.format(now,str((max_val_acc*100).round(2))), [max_val_acc], fmt='%f')
        stats = str(now) + ' ' + str((max_val_acc*100).round(2)) + '\n'
        with open('model-comparison/best-models.txt', 'a') as f:
            f.write(stats)
    plt.show()

In [9]:
def EvaluateModel(model, test_set, str1, now, save = True):
    
    print('\n PREDICTING LABELS OF TEST IMAGES')
    result = model.predict(test_set)
    y_pred = np.argmax(result, axis=1)
    
    if save==True:
        #save y_pred to csv file
        os.mkdir('model-comparison/'+now+'/'+str1)
        np.savetxt('model-comparison/{}/{}/pred.csv'.format(now,str1), y_pred, delimiter=',', fmt='%d')
    
    y_true = test_set.classes # List containing true labels for each image.

    # Understanding classification power of model on each class    
    report = classification_report(y_true, y_pred, target_names=test_set.class_indices.keys())
    report_d = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True, target_names=test_set.class_indices.keys())).transpose()
    report_d['support']['accuracy'] = report_d['support']['macro avg']

    annot = report_d.copy()
    annot.iloc[:, 0:3] = (annot.iloc[:, 0:3]*100).applymap('{:.2f}'.format) + ' %'
    annot.iloc[7, 1] = ''
    annot.iloc[7, 0] = ''
    annot['support'] = annot['support'].astype(int)

    # how to save report as image
    norm = Normalize(-1,1)
    cmap = LinearSegmentedColormap.from_list("", [[norm(-1.0), "white"],[norm( 1.0), "white"]])
    plot = sns.heatmap(report_d, annot=annot, cmap=cmap, cbar=False, fmt='')
    fig = plot.get_figure()
    if save==True:
        fig.savefig('model-comparison/{}/{}/report.png'.format(now,str1))
    
    f1_score = ((report_d['f1-score']['weighted avg']*100000//10)/100)
    accuracy = ((report_d['f1-score']['accuracy']*100000//10)/100)
    print('\nAccuracy of model prediction is: {:.2f} %'.format(accuracy))
    print('\nF1-score of model prediction is: {:.2f} %'.format(f1_score))

    cm = confusion_matrix(y_true, y_pred)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                              display_labels=test_set.class_indices.keys()
                              )
    disp.plot(cmap='Reds')
    disp.ax_.set_title('Confusion Matrix')
    plt.show()
    if save==True:
        disp.figure_.savefig('model-comparison/{}/{}/cm.png'.format(now,str1))

In [10]:
def train_new_model(model):
    # Extra
    #class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(train_set.classes), y=train_set.classes)
    #class_weights_dict=dict(zip(np.unique(train_set.classes),class_weights))
    #keras.utils.set_random_seed(42)     
    # inside model.fit: class_weight=class_weights_dict,

    # Train new model and evaluate
    now = datetime.datetime.now().strftime("%d-%m-%H-%M")
    os.mkdir('model-comparison/'+now)
    def myprint(s):
        with open(f'model-comparison/{now}/modelsummary.txt','a') as f:
            print(s, file=f)
    model.summary(print_fn=myprint)
    with open('model-comparison/last.txt', 'w') as f:
        f.write(str(now))
    return now

In [12]:
# Transfer Learning

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

base_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0],target_size[1],3))
                                   
for layer in base_model.layers:
    layer.trainable = True
    
# Add layers at the end
model = base_model.output
model = Flatten()(model)

model = Dense(512, kernel_initializer='he_uniform')(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)

model = Dense(128, kernel_initializer='he_uniform')(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)

model = Dense(32, kernel_initializer='he_uniform')(model)
model = Dropout(0.2)(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)

output = Dense(len(labels), activation='softmax')(model)

model = Model(inputs=base_model.input, outputs=output)

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])
n_epoch = 10

In [13]:
now = train_new_model(model)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto')
checkpoint = ModelCheckpoint(f"model-comparison/{now}/model.h5", monitor='val_accuracy', save_best_only=True, mode='max')

history = model.fit(train_set,
                    epochs=60,
                    callbacks=[reduce_lr,early_stop,checkpoint],
                    validation_data=val_set,
                    shuffle=True)


# Evaluate Transfer Learning model
loss_plot(model, history.history, now)
EvaluateModel(model, val_set, 'val', now)
EvaluateModel(model, test_set, 'test', now)
os.system(finish_sound)

Epoch 1/60


2023-12-18 02:24:03.496138: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


81/81 [==============================] - 75s 885ms/step - loss: 1.5845 - accuracy: 0.3463 - val_loss: 1.6928 - val_accuracy: 0.1784 - lr: 1.0000e-04
Epoch 2/60
81/81 [==============================] - 70s 865ms/step - loss: 1.1114 - accuracy: 0.5857 - val_loss: 1.8091 - val_accuracy: 0.2162 - lr: 1.0000e-04
Epoch 3/60
81/81 [==============================] - 73s 905ms/step - loss: 0.8053 - accuracy: 0.7630 - val_loss: 3.0241 - val_accuracy: 0.2162 - lr: 1.0000e-04
Epoch 4/60
81/81 [==============================] - 71s 870ms/step - loss: 0.5911 - accuracy: 0.8525 - val_loss: 1.7226 - val_accuracy: 0.2459 - lr: 1.0000e-04
Epoch 5/60
81/81 [==============================] - 76s 940ms/step - loss: 0.4378 - accuracy: 0.9117 - val_loss: 1.6872 - val_accuracy: 0.2838 - lr: 1.0000e-04
Epoch 6/60
29/81 [=========>....................] - ETA: 50s - loss: 0.3402 - accuracy: 0.9524

KeyboardInterrupt: 